<a href="https://colab.research.google.com/github/kadeng/pykurs/blob/main/notebooks/python_kurs_2_text_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Kurs

Dieses Notebook ist Teil 2, eines kleinen Kurses mit Übungen, in dem auf einige Themen mittleren bis einfachen Komplexitätsgrads eingegangen wird, die in generellen Einführungen zu Python manchmal zu kurz kommen, in der Praxis aber relevant sind.


### Text Parsing & Manipulation

in diesem Teil des Kurses, lernen wir [Text Parsing](https://en.wikipedia.org/wiki/Parsing). Das ist in diesem Kontext definiert als das interpretieren eines Textes anhand von formellen Regeln, so dass am Ende eine maschineninterpretierbare Datenstruktur entsteht welche die Bedeutung des Texts korrekt wiedergibt.

Wir beginnen mit sehr simplen Text-Parsing aufgaben, wie z.B. dem **zerlegen von Text** anhand von Separatoren und dem **Bereinigen und Normalisieren** durch einfache Textersetzungen.

Anschliessend widmen wir uns einigen **standardisierten Formaten zu**: CSV, JSON und YAML.

Anschliessend gehen wir auf das Thema **Regulaere Ausdruecke ( Regular Expressions )** ein, mit denen bereits sehr anspruchsvolle Aufgaben ( Muster- und Teilmusterkennung ) ausgefuehrt werden koennen.

Abschliessend gehen wir darauf ein, wie man durch **mehrstufiges Parsing** auch Texte interpretiert die man mit einem einzelnen Regulaeren Ausdruck nicht interpretiert bekommt.


### Manuelles Parsing

Im folgenden sehen wir einige Beispiele für Parsing von kurzen relativ leicht zu zerlegenden und interpretierenden Texten

In [154]:
# Interpretiere einen Text als eine ganze Zahl

a = " 23"
int(a)

23

In [155]:
# Geht das?
a = "Hallo?"
int(a)

ValueError: ignored

In [158]:
# Beispiel fuer einen simplen Parser mit Ausnahmebehandlung
def parse_as_int(s, default_value=-1):
  try:
    return int(s)
  except:
    return default_value

print(parse_as_int("23"))
print(parse_as_int("Hallo", "Hallo ist keine Zahl"))

23
Hallo ist keine Zahl


In [167]:
# Floating-Point Zahl parsen?
def parse_as_float(s, default_value=float('NaN')):
  try:
    return float(s)
  except:
    return default_value

print(parse_as_float("23.4"))
print(parse_as_float("1e-2"))
print(parse_as_float("Hallo"))
print(type(parse_as_float("Hallo"))) # nan ist ein float-wert der als "keine Zahl" interpretiert wird ( NaN = Not a Number)

23.4
0.01
nan
<class 'float'>


In [169]:
# Kategorische Werte parsen?

color_code_map = {
    "red" : 1,
    "blue" : 2,
    "unknown" : 0
}

def parse_color_code(s):
  return color_code_map.get(s, 0)

print(parse_color_code("red"))
print(parse_color_code("blue"))
print(parse_color_code("Blue")) # klappt nicht, weil B grossgeschrieben
print(parse_color_code("Blue".lower())) # normalisiert klappt das

1
2
0
2


In [170]:
# Jetzt ein komplexeres Beispiel
zeile = "12;239$;C23: Yes, C24:No;239"
werte = zeile.split(";")
print(werte)

['12', '239$', 'C23: Yes, C24:No', '239']


In [161]:
# Man kann auch gleich den Rückgabewert in mehrere Variablen **entpacken** wenn man weiss dass es immmer eine feste Anzahl ist
substance_id, price, results, study = zeile.split(";")
print(f"substance_id={substance_id}, results={results}")

substance_id=12, results=C23: Yes, C24:No


In [162]:
# Vielleicht wollen wir aber auch etwas weiter zerlegen?

def parse_results_str(results_str : str) -> dict[str,str]:
    """
    Zerlegt einen einzelnen results_str wie "C23: Yes, C24:No"
    """
    results = {} # rueckgabewert dict das wir gleich befuellen wollen
    teilergebnisse_liste = results_str.split(",")
    # jetzt muesste teilergebnisse_liste sowas wie ["C23: Yes", "C24:No" ] sein.
    # zerlegen wir jeden Eintrag weiter
    for teilergebnis in teilergebnisse_liste:
      # teilergebnis ist sowas wie "C23: Yes"
      key, value = teilergebnis.split(":") # gibt uns zwei Werte
      key = key.strip() # leerzeichen am anfang und ende entfernen
      value = value.strip().lower() # leerzeichen am anfang und ende entfernen und kleinschreiben
      # Sicherheitspruefung: Wir nehmen an, dass jeder key nur einmal vorkommt. Sonst Fehlermeldung
      assert key not in results, f"Eintrag {key} ist doppelt vorhanden"
      # jetzt koennen wir das in das Resultat packen
      results[key] = value

    return results # alles zurueckgeben


def parse_zeile(zeile):
    """
    Parst eine Zeile wie "12;239$;C23: Yes, C24:No;239"
    """
    substance_id_str, price_str, results_str, study_str = zeile.split(";")
    price = float(price_str.replace("$", "")) # Dollarzeichen entfernen und in float konvertieren
    substance_id = int(substance_id_str) # in Integer konvertieren
    study_id = int(study_str)
    results_dict : dict[str,str] = parse_results_str(results_str)
    resultate = {
        "substance_id" : substance_id,
        "study_id" : study_id,
        "results" : results_dict,
        "price" : price
    }
    return resultate



In [163]:
# Jetzt mal ausfuehren
parse_zeile(zeile)


{'substance_id': 12,
 'study_id': 239,
 'results': {'C23': 'yes', 'C24': 'no'},
 'price': 239.0}

#### Uebungen

* Schreib die Funktionen oben so um, dass anstatt von "yes" oder "no" ein Wert von **True** oder **False** zurueckgebeben wird in den Detail-Results.
* Erzeuge ein paar Varianten fuer input-Zeilen und pass ggf. den Code an um mit diesen Inputs klarzukommen.
* Bau eine Fallunterscheidung ein: Wenn ein bestimmter Text im String vorkommt, verwende den einen Parser, wenn er nicht vorkommt einen anderen. (Dazu kannst du das "in" Keyword verwenden, siehe Teil 1 )



### CSV, JSON & YAML

CSV, JSON und YAML sind gaengige textbasierte Datenformate die gut Menschen- und Maschinenlesbar sind.

Das obige Resultat laesst sich sehr gut in JSON oder YAML uebersetzen, um es in besser Maschinenlesbarer Form abzulegen:



In [ ]:
import json
import yaml

In [177]:
# Aus dem geparsten Dictionary einen String im JSON Format machen, mit Hilfe von json.dumps(...)
json_str = json.dumps(parse_zeile(zeile), indent=2)
json_str

'{\n  "substance_id": 12,\n  "study_id": 239,\n  "results": {\n    "C23": "yes",\n    "C24": "no"\n  },\n  "price": 239.0\n}'

In [178]:
print(json_str)

{
  "substance_id": 12,
  "study_id": 239,
  "results": {
    "C23": "yes",
    "C24": "no"
  },
  "price": 239.0
}


In [184]:
# it's just a string, see
print(json_str[10:20])

ance_id": 


In [182]:
# Das schoene ist, wir koennten den String speichern, spaeter laden und dann die Daten wiederherstellen, einfach so
# mit json.loads
result = json.loads(json_str)

In [183]:
# Das hier ist kein String
result["price"]

239.0

In [214]:
# ganz aehnlich geht das mit dem YAML parser / generator
# der etwas leichter zu lesen ist
yaml_str = yaml.safe_dump(parse_zeile(zeile), indent=2)
print(yaml_str)

price: 239.0
results:
  C23: 'yes'
  C24: 'no'
study_id: 239
substance_id: 12



In [215]:
# Das YAML kann man auch automatisch parsen lassen
result2 = yaml.safe_load(yaml_str)
print(result2)

{'price': 239.0, 'results': {'C23': 'yes', 'C24': 'no'}, 'study_id': 239, 'substance_id': 12}


#### CSV

Das vermutlich einfachste Format ist CSV, was für "Comma Separated Values" steht. Es repräsentiert Tabellen einfach als Textdatei mit einer Textzeile pro Tabellenzeile, wobei jede Spalte durch einen Separator getrennt wird ( typischerweise durch Komma, Tab-Charakter oder Semikolon ).

Das CSV Format kann in verschiedenen Varianten auftreten, bei denen z.B. die Werte manchmal in Anführungszeichen gesetzt werden oder nicht, mit Headern oder ohne usw.

Das hier ist ein ganz typisches CSV:

```
CUSTOMER_ID;ORDER_ID;UNIT_PRICE;AMOUNT;NAME
1;32832;28.4$;4;iPad Schutzhülle A9
2;32833;9.99$;4;Panzerglas Huawei Mate 20
```
usw..

alternativ ist aber auch dies eine CSV

```
"customer id","order id","unit price","amount","name"
1;32832;"28.4$";4;"iPad Schutzhülle A9"
2;32833;"9.99$";4;"Panzerglas, Huawei Mate 20"
```

in diesem Beispiel zeigt sich auch direkt eines der gängigen Probleme dieser Formate: Wie geht man mit Zeichen um, die eine besondere Bedeutung haben? Im Feld "Panzerglas, Huawei Mate 20" findet sich ein Komma. Das ist nur deshalb kein Problem, weil der Text in Anführungszeichen ist. Dennoch kann man jetzt nicht mehr einfach die Zeile anhand von Kommas trennen um an alle Werte zu gelangen. Ein richtiger **CSV Parser** muss her.

Glücklicherweise gibt es ja **pandas**...

In [187]:
# Beginnen wir mit dem Import
import pandas
import io

In [209]:
csv_text = """
"customer id";"order id";"unit price";"amount";"name"
1;32832;"28.4$";4;"iPad Schutzhülle A9"
2;32833;"9.99$";4;"Panzerglas; Huawei Mate 20"
"""

csv_io = io.StringIO(csv_text.strip()) # das brauchen wir, weil es nicht aus einer Datei kommt

df = pandas.read_csv(csv_io, sep=";")
df

,customer id,order id,unit price,amount,name
0,1,32832,28.4$,4,iPad Schutzhülle A9
1,2,32833,9.99$,4,Panzerglas; Huawei Mate 20
